# Heart classification
The data set from the kaggle:https://www.kaggle.com/datasets/fedesorian/heart-failure-prediction
### imports
-here we import torch for gradients and networks<br>
-pandas for data processing<br>
-sci-kit learn for its usefull functions<br>
-shuffle for to shuffle the data 


In [49]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

### Reading the csv file
-next  we shuffle the data using the shuffle function<br> -printing the first 5 rows 

In [50]:
features=pd.read_csv("./heart.csv")
features=shuffle(features)
features.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
295,63,1,0,140,187,0,0,144,1,4.0,2,2,3,0
235,51,1,0,140,299,0,1,173,1,1.6,2,0,3,0
202,58,1,0,150,270,0,0,111,1,0.8,2,0,3,0
175,40,1,0,110,167,0,0,114,1,2.0,1,0,3,0
115,37,0,2,120,215,0,1,170,0,0.0,2,0,2,1


#### next popping out the label

In [51]:
labels=features.pop("output")

### convertions
-converting the dataframes into numpy arrays

In [52]:
features=np.array(features)
lables=np.array(labels).reshape(-1,1)

* next line  we will normalize the features so the training process goes faster and printing the first index of the features 

In [53]:
features = ss().fit_transform(features)
features[0]

array([ 0.9521966 ,  0.68100522, -0.93851463,  0.47839125, -1.14530589,
       -0.41763453, -1.00583187, -0.24694024,  1.43548113,  2.55392051,
        0.97635214,  1.24459328,  1.12302895])

* converting the arrays into tensors so we can perform ml operations on them<br>
* finally we are printing the shapes of features and labels

In [54]:
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32).reshape(-1,1)
print(features.shape)
print(labels.shape)

torch.Size([303, 13])
torch.Size([303, 1])


#### -creating a reasonally layered model with forst input size as 16 followed by sigmoid function 
* sigmoid function output always in between 0 and 1

In [55]:
model = torch.nn.Sequential(
    torch.nn.Linear(13,9),
    torch.nn.ReLU(),
    torch.nn.Linear(9,3),
    torch.nn.ReLU(),
    torch.nn.Linear(3,1),
    torch.nn.Sigmoid()
)

#####  the loss we will use is Binary cross entropy
##### the optimizer we will use is SGD-(stochastic gradient descent) to update network weights during training

In [56]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.SGD(model.parameters(), lr=1e-3)

##### here the training loop for 10000 epochs
##### we simply make predictions and calculating the loss
##### i next step we will find gradients and optimize the model to reduce loss
### For finding the Accuracy 
##### if predictions are greater than 0.5 then we will make the predictions equal to 1
##### if less than 0.5 then we will make the predictions equal to 0
##### now ,if the predictions and labels are equal then we increment the count 
##### now ,accuracy is equal to total count devides length of the predictions

In [57]:
epochs = 100000

for epoch in range(epochs):
    preds = model(features)
    loss = loss_fn(preds,labels)

    loss.backward()

    opt.step()
    opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for i in range(len(preds)):
                if preds[i][0] >= 0.5:
                    preds[i][0] = 1
                else:
                    preds[i][0] = 0

                if preds[i][0] == labels[i][0]:
                    right += 1

            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ len(preds), 2)}%")


Loss : 0.3979076147079468
Accuracy : 84.49%
Loss : 0.3210539221763611
Accuracy : 87.79%
Loss : 0.29974445700645447
Accuracy : 88.45%
Loss : 0.2838155925273895
Accuracy : 87.79%
Loss : 0.27076682448387146
Accuracy : 88.78%
Loss : 0.25570034980773926
Accuracy : 89.11%
Loss : 0.2382168173789978
Accuracy : 90.76%
Loss : 0.21924343705177307
Accuracy : 91.42%
Loss : 0.1985958069562912
Accuracy : 92.08%
Loss : 0.1803819239139557
Accuracy : 92.74%


#### Here Saving the weights and biases of our model so we can load em up again whenever we want to make a prediction.

In [ ]:
torch.save(model.state_dict(),"heart_classification.pth")